In [ ]:
model noble_1962__membrane(V, i_Na, i_K, i_Leak)

  // Rate Rules:
  V' = -(i_Na + i_K + i_Leak)/Cm;

  // Variable initializations:
  V = -87;
  Cm = 12;
  species i_Na, i_K, i_Leak; // input
end

model noble_1962__sodium_channel_m_gate(m, V)
  alpha_m = 100*(-V - 48) / ( exp((-V - 48)/15) - 1 )
  beta_m = 120*(V+8)/( exp((V+8)/5) - 1 )
  m' = alpha_m*(1-m) - beta_m*m
  // Variable initializations:
  m = 0.01; // output
  species V; // input
end

model noble_1962__sodium_channel_h_gate(h, V)
  alpha_h = 170*exp((-V-90)/20)
  beta_h = 1000/(1 + exp((-V-42)/10))
  h' = alpha_h*(1-h) - beta_h*h
  // Variable initializations:
  h = 0.8; // output
  species V; // input
end

model noble_1962__sodium_channel(i_Na, V, m, h)
  // Sub-modules, and any changes to those submodules:
  sodium_channel_m_gate: noble_1962__sodium_channel_m_gate(m, V, time_);
  sodium_channel_h_gate: noble_1962__sodium_channel_h_gate(h, V, time_);
  
  // Assignment Rules:
  i_Na := (g_Na + 140)*(V - E_Na);
  g_Na := power(m, 3)*h*g_Na_max;

  // Variable initializations:
  g_Na_max = 400000; // us
  E_Na = 40; // mV
end

model noble_1962__potassium_channel_n_gate(n, V)
  alpha_n = 0.1*(-V-50) / ((exp(-V-50)/10) - 1)
  beta_n = 2*exp((-V-90)/80)
  n' = alpha_n*(1-n)-beta_n*n
  // Variable initializations:
  n = 0.01; // output
  species V; // input
end

model noble_1962__potassium_channel(i_K, V, n)
  i_K = (g_K1 + g_K2)*(V+100)
  g_K1 = 1200*exp((-v-90)/50) + 15*exp((v+90)/60)
  g_K2 = 1200*n^4
  // Sub-modules, and any changes to those submodules:
  potassium_channel_n_gate: noble_1962__potassium_channel_n_gate(n, V);

  // Variable initializations:
  species V; // output
  species n = 0.01; // input
end

model noble_1962__leakage_current(i_Leak, V)
  i_Leak = g_L*(V-E_L)
  // Variable initializations:
  g_L = 75;
  E_L = -60;
  species V; // input
end

model *noble_1962____main()
  // Sub-modules, and any changes to those submodules:
  membrane: noble_1962__membrane(V, i_Na, i_K, i_Leak);
  sodium_channel: noble_1962__sodium_channel(i_Na, V, m, h);
  potassium_channel: noble_1962__potassium_channel(i_K, V, n);
  leakage_current: noble_1962__leakage_current(i_Leak, V);
end

Noble62: noble_1962____main();